# National Model 1a

- Equifax, TU, Experian National model 
- Features selection on national data
- Time split on Equifax

The notebook covers all three steps for creatign a new national model; feature selection, validation model and fold-in model. depending on the applied use case, not each step may be necessary (e.g., if the feature selection does not change compaed to the previous interation, step 1 may be skipped, etc.

In [1]:
import pandas as pd
import numpy as np

import model_engine
model_engine.__version__

'v1.13.0'

In [2]:
from model_engine.power.post_sale import NationalModelBuilder
from model_engine.io.loaders import load_json

import json

/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_modeling/lib/python3.1

In [3]:
import os
import numba as nb
n_jobs = os.getenv('CPU_LIMIT', default=-1)
n_jobs = 20
print(n_jobs)
nb.set_num_threads(n_jobs)

20


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
input_client_name = 'penfed'
input_project_name = 'autoloan'
input_bureau_name = 'normalized'
model_type = 'standard_model_member_data' # options are 'standard_model', 'mega_model', 'standard_model_member_data', 'standard_model_ln_data', ''- None defaults to 'standard_model'

print(input_client_name, input_project_name, input_bureau_name, model_type)

# national_model_id = 'nationalmodel1b'

penfed autoloan normalized standard_model_member_data


## Create a national model

In [6]:
# client' feature selection model

feature_selection_model_id = 'nationalmodel2a'

In [7]:
feature_selection_model_output_path = f'/d/shared/users/lyt/penfed_poc1_national_models/{feature_selection_model_id}'
feature_selection_model_output_path

'/d/shared/users/lyt/penfed_poc1_national_models/nationalmodel2a'

In [8]:
config = {
  "model_type": "standard_model",
  "storage_location": "s3",
  "lookalike": {
    "bureau": "normalized",
    "accounts":[
      "auto"
    ],
    "splits": [
      "train",
      "valid",
      "test"
    ],
    "pull_names": None,
    "states": None,
    "max_row": None,
    "tables": ['trade', 'inq', 'bnkr', 'collec', 'target', 'app', 'member'],
  },
  "model_config": {
      "data_split": {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2021-04-01"
          },
          "valid": {
            "start_date": "2021-04-01",
            "end_date": "2021-07-01"
          },
          "test": {
            "start_date": "2021-07-01",
            "end_date": "2022-01-01"
          }
    },
    "target": "final_DQ60_m24",
    "base_features": f'{feature_selection_model_output_path}/keep_features.json',
#    "feature_filter_rules": "/home/lyt/CODEBASE/model-engine_penfed/model-engine/model_engine/assets/power/feature_rules/intuitive_features.json",
    "memory_efficient": True,
    "pipeline_factory": None,
    "bivariate_fe_instructions": [],
    "monotonic_constraints_list": [],
    "exclusion_list": [],
    "feature_definition_list": [],
    "key_factor_mapping_list": [],
    "fold_valid": True,
    "mandatory_features": []
  }
}

In [9]:
national_model_id = 'nationalmodel2b'
national_model_output_path = f'/d/shared/users/lyt/penfed_poc1_national_models/{national_model_id}'


In [10]:
national_model_output_path

'/d/shared/users/lyt/penfed_poc1_national_models/nationalmodel2b'

In [ ]:
nmb = NationalModelBuilder(
    configuration=config,
    model_output_path=national_model_output_path,
    overwrite_model_output=True
)
nmb.run()

Attempting to create a national model with full data split here /d/shared/users/lyt/penfed_poc1_national_models/nationalmodel2b/supporting_model
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4



/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify

parsing model-builder asset







/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should s

Configuring model builder


...

INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <n_tiers>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <n_tiers>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing Input

building model


INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.459946
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000937
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000573
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_exclusion_list>, total time spent: 0:00:00.000574
INFO:zaml.artifact_engine.logger:Executing BivariateFeParser <parsed_biv_fe_instructions>...
INFO:zaml.artifact_engine.logger:Finished <parsed_biv_fe_instructions>, total time spent: 0:00:00.000604
INFO:zaml.artifact_engine.logger:Executing NeutralizationListParser <parsed_neutralization_list>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifa

-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 2.180s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.016s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.016s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.017s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.023s
-------------------------
Name: trade
Transformer type: None
Number of features: 216
Time spent: 0.034s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 1.734s
-------------------------
Name: bnkr FE
Transformer type: ManyToOneEngine
Number of features: 8
Time spent: 0.083s
-------------------------
Name: collec FE
Transformer type: ManyToOneEngine
Number of features: 5
Time spent: 0.081s
-------------------------
Name: inq FE
Tr

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 270
Time spent: 129.086s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 270
Time spent: 0.003s


In [12]:
ftr_importance = pd.read_parquet(f'/d/shared/users/lyt/penfed_poc1_national_models/{national_model_id}/feature_importance.parquet')
ftr_importance.head(10)

,feature,importance
0,inq_inquiries__sum,0.008055
1,trade_months_since_openDate__mean_by_acctType_...,0.006516
2,trade_blncAmt__mean_by_acctType_auto,0.006265
3,inq_inquiries__count_by_indCode_auto_mortgage,0.005952
4,trade_hcAmt_accts_never_dq__sum,0.005712
5,trade_clAmt__mean_by_prtfType_revolving,0.005580
6,trade_months_since_rptDate__mean_by_acctType_c...,0.005531
7,trade_blnc_to_hc_all_non_derog_accts__mean_by_...,0.005528
8,trade_hcAmt_accts_with_rcntPmt__sum,0.005388
9,trade_months_since_rptDate__sum_by_acctType_auto,0.005368
